GOOGLE COLLAB LINK=https://colab.research.google.com/drive/16pkEkSkoHGbmOuRg6hgS4L_N0RRepYZF?usp=sharing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ Libraries imported successfully")
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
print("\n📂 Loading datasets...")

In [ ]:
sentiment_df = pd.read_csv('/content/fear_greed_index.csv')
trades_df = pd.read_csv('/content/historical_data.csv')

In [ ]:
print("\n" + "="*80)
print("DATA EXPLORATION")
print("="*80)
print("\n📊 SENTIMENT DATA OVERVIEW:")
print(sentiment_df.head())
print("\nColumns:", sentiment_df.columns.tolist())
print("\nData types:\n", sentiment_df.dtypes)
print("\nMissing values:\n", sentiment_df.isnull().sum())


In [ ]:
print("\n📈 TRADER DATA STATISTICS:")
print(trades_df.describe())

In [ ]:
print("\n" + "="*80)
print("DATA PREPROCESSING")
print("="*80)


# Parse dates
sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
trades_df['Timestamp'] = pd.to_datetime(trades_df['Timestamp'])

# Extract date from timestamp for merging
trades_df['date'] = trades_df['Timestamp'].dt.date
sentiment_df['date'] = sentiment_df['date'].dt.date

# Handle missing values in critical columns
print("\n🔧 Handling missing values...")
initial_rows = len(trades_df)
trades_df = trades_df.dropna(subset=['closedPnL'])

# Fill missing leverage with median
if trades_df['leverage'].isnull().sum() > 0:
    median_leverage = trades_df['leverage'].median()
    trades_df['leverage'].fillna(median_leverage, inplace=True)

print(f"Rows after cleaning: {len(trades_df)} (removed {initial_rows - len(trades_df)} rows)")


In [ ]:
print("\n" + "="*80)
print("DATA PREPROCESSING")
print("="*80)


# Parse dates
sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
trades_df['Timestamp'] = pd.to_datetime(trades_df['Timestamp'], unit='s')

# Extract date from timestamp for merging
trades_df['date'] = trades_df['Timestamp'].dt.date
sentiment_df['date'] = sentiment_df['date'].dt.date

# Handle missing values in critical columns
print("\n🔧 Handling missing values...")
initial_rows = len(trades_df)
trades_df = trades_df.dropna(subset=['Closed PnL'])

print(f"Rows after cleaning: {len(trades_df)} (removed {initial_rows - len(trades_df)} rows)")

In [ ]:
def remove_outliers_iqr(df, column, multiplier=1.5):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

print("\n🔧 Removing outliers...")
before_outlier_removal = len(trades_df)
trades_df = remove_outliers_iqr(trades_df, 'Closed PnL', multiplier=3)

print(f"Rows after outlier removal: {len(trades_df)} (removed {before_outlier_removal - len(trades_df)} outliers)")

In [ ]:
print("\n🔗 Merging sentiment and trader data...")
merged_df = trades_df.merge(
    sentiment_df[['date', 'classification']],
    on='date',
    how='left'
)

print(f"✓ Merged dataset: {merged_df.shape[0]} rows")
print(f"Rows with sentiment data: {merged_df['classification'].notna().sum()}")

In [ ]:
import os

# Create additional features
print("\n🎯 Feature engineering...")

# Profitable trade indicator
merged_df['is_profitable'] = (merged_df['Closed PnL'] > 0).astype(int)

# Absolute PnL
merged_df['abs_pnl'] = merged_df['Closed PnL'].abs()

# Trade value
merged_df['trade_value'] = merged_df['Size Tokens'] * merged_df['Execution Price']

# Hour of day (for temporal analysis)
merged_df['hour'] = pd.to_datetime(merged_df['Timestamp']).dt.hour

# Day of week
merged_df['day_of_week'] = pd.to_datetime(merged_df['Timestamp']).dt.dayofweek

print("✓ Feature engineering complete")

# Save processed data
output_dir = 'csv_files'
os.makedirs(output_dir, exist_ok=True)
merged_df.to_csv(f'{output_dir}/processed_data.csv', index=False)
print(f"\n💾 Processed data saved to {output_dir}/processed_data.csv")

In [ ]:
print("\n" + "="*80)
print("SENTIMENT ANALYSIS")
print("="*80)

# Sentiment distribution
sentiment_counts = sentiment_df['classification'].value_counts()
sentiment_pct = sentiment_df['classification'].value_counts(normalize=True) * 100

print("\n📊 Sentiment Distribution:")
for sentiment, count in sentiment_counts.items():
    pct = sentiment_pct[sentiment]
    print(f"  {sentiment}: {count} days ({pct:.1f}%)")

In [ ]:
import os

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart
axes[0].pie(sentiment_counts.values, labels=sentiment_counts.index, autopct='%1.1f%%',
            colors=sns.color_palette("husl", len(sentiment_counts)), startangle=90)
axes[0].set_title('Market Sentiment Distribution', fontsize=14, fontweight='bold')

# Time series
axes[1].plot(sentiment_df['date'], sentiment_df['value'], linewidth=1, alpha=0.7)
axes[1].set_title('Sentiment Value Over Time', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Sentiment Value')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()

# Create the 'outputs' directory if it doesn't exist
output_dir = 'outputs'
os.makedirs(output_dir, exist_ok=True)

plt.savefig(f'{output_dir}/sentiment_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Sentiment visualizations saved")

In [ ]:
print("\n" + "="*80)
print("TRADER BEHAVIOR ANALYSIS")
print("="*80)

# Overall metrics
print("\n📊 OVERALL TRADING METRICS:")
print(f"Total Trades: {len(merged_df):,}")
print(f"Unique Traders: {merged_df['Account'].nunique():,}")
print(f"Total PnL: ${merged_df['Closed PnL'].sum():,.2f}")
print(f"Average PnL per Trade: ${merged_df['Closed PnL'].mean():,.2f}")
print(f"Median PnL per Trade: ${merged_df['Closed PnL'].median():,.2f}")
print(f"Win Rate: {(merged_df['is_profitable'].mean() * 100):.2f}%")
print(f"Average Trade Size: ${merged_df['trade_value'].mean():,.2f}")

In [ ]:
print("\n📊 METRICS BY SENTIMENT:")
sentiment_metrics = merged_df.groupby('classification').agg({
    'Closed PnL': ['sum', 'mean', 'median', 'std'],
    'trade_value': ['sum', 'mean'],
    'is_profitable': 'mean',
    'Account': 'nunique'
}).round(2)

display(sentiment_metrics)

In [ ]:
print("\n" + "="*80)
print("STATISTICAL ANALYSIS")
print("="*80)

# Prepare data for testing
fear_data = merged_df[merged_df['classification'] == 'Fear']
greed_data = merged_df[merged_df['classification'] == 'Greed']

# T-test: PnL comparison
print("\n🔬 T-TEST: PnL in Fear vs Greed")
t_stat_pnl, p_value_pnl = stats.ttest_ind(
    fear_data['Closed PnL'].dropna(),
    greed_data['Closed PnL'].dropna()
)
print(f"T-statistic: {t_stat_pnl:.4f}")
print(f"P-value: {p_value_pnl:.4f}")
print(f"Result: {'Significant' if p_value_pnl < 0.05 else 'Not significant'} difference at α=0.05")

# Chi-square test: Side preference by sentiment
print("\n🔬 CHI-SQUARE TEST: Side Preference by Sentiment")
contingency_table = pd.crosstab(merged_df['Side'], merged_df['classification'])
print("\nContingency Table:")
display(contingency_table)

chi2, p_value_chi, dof, expected = stats.chi2_contingency(contingency_table)
print(f"\nChi-square statistic: {chi2:.4f}")
print(f"P-value: {p_value_chi:.4f}")
print(f"Degrees of freedom: {dof}")
print(f"Result: {'Significant' if p_value_chi < 0.05 else 'Not significant'} association at α=0.05")

# Correlation analysis
print("\n🔬 CORRELATION ANALYSIS:")
correlation_features = ['Closed PnL', 'trade_value', 'Size Tokens']
correlation_matrix = merged_df[correlation_features].corr()
display(correlation_matrix)

In [ ]:
print("\n" + "="*80)
print("TRADES DATA PREPROCESSING (CONSOLIDATED)")
print("="*80)

# Load the original trades data again to ensure correct starting point
trades_df = pd.read_csv('/content/historical_data.csv')

# Parse dates with unit='ms'
trades_df['Timestamp'] = pd.to_datetime(trades_df['Timestamp'], unit='ms')

# Extract date from timestamp for merging
trades_df['date'] = trades_df['Timestamp'].dt.date

# Handle missing values in critical columns
print("\n🔧 Handling missing values...")
initial_rows = len(trades_df)
trades_df = trades_df.dropna(subset=['Closed PnL'])
print(f"Rows after cleaning: {len(trades_df)} (removed {initial_rows - len(trades_df)} rows)")

# Remove outliers
print("\n🔧 Removing outliers...")
before_outlier_removal = len(trades_df)
trades_df = remove_outliers_iqr(trades_df, 'Closed PnL', multiplier=3)
print(f"Rows after outlier removal: {len(trades_df)} (removed {before_outlier_removal - len(trades_df)} outliers)")

print("\n✓ Trades data preprocessing complete")

print("\nDate range in trades_df after consolidated preprocessing:")
if not trades_df.empty:
    print(f"Date Range: {trades_df['date'].min()} to {trades_df['date'].max()}")
    print("Sample Unique Dates:")
    display(trades_df['date'].unique()[:10])
else:
    print("trades_df is empty after preprocessing.")

In [ ]:
print("Unique dates in trades_df after preprocessing:")
display(trades_df['date'].unique())

print("\nUnique dates in sentiment_df after preprocessing:")
display(sentiment_df['date'].unique())

In [ ]:
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Visualization 1: PnL by Sentiment
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Bar plot: Total PnL
pnl_by_sentiment = merged_df.groupby('classification')['Closed PnL'].sum()
colors = ['#ff6b6b' if x == 'Fear' else '#51cf66' for x in pnl_by_sentiment.index]
axes[0].bar(pnl_by_sentiment.index, pnl_by_sentiment.values, color=colors, alpha=0.7, edgecolor='black')
axes[0].set_title('Total PnL by Market Sentiment', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Total PnL ($)')
axes[0].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(pnl_by_sentiment.values):
    axes[0].text(i, v, f'${v:,.0f}', ha='center', va='bottom' if v > 0 else 'top', fontweight='bold')

# Box plot: PnL distribution
merged_df.boxplot(column='Closed PnL', by='classification', ax=axes[1], patch_artist=True)
axes[1].set_title('PnL Distribution by Sentiment', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Market Sentiment')
axes[1].set_ylabel('Closed PnL ($)')
plt.suptitle('')  # Remove default title

plt.tight_layout()
plt.savefig('outputs/pnl_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ PnL analysis saved")

# Visualization 2: Correlation Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix of Trading Metrics', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('outputs/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Correlation matrix saved")

# Visualization 3: Trading Volume by Sentiment
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Total volume
volume_by_sentiment = merged_df.groupby('classification')['trade_value'].sum()
axes[0, 0].bar(volume_by_sentiment.index, volume_by_sentiment.values, color=colors, alpha=0.7, edgecolor='black')
axes[0, 0].set_title('Total Trading Volume by Sentiment', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Volume ($)')
axes[0, 0].grid(True, alpha=0.3, axis='y')

# Number of trades
trades_by_sentiment = merged_df.groupby('classification').size()
axes[0, 1].bar(trades_by_sentiment.index, trades_by_sentiment.values, color=colors, alpha=0.7, edgecolor='black')
axes[0, 1].set_title('Number of Trades by Sentiment', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Count')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Win rate
win_rate_by_sentiment = merged_df.groupby('classification')['is_profitable'].mean() * 100
axes[1, 0].bar(win_rate_by_sentiment.index, win_rate_by_sentiment.values, color=colors, alpha=0.7, edgecolor='black')
axes[1, 0].set_title('Win Rate by Sentiment', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Win Rate (%)')
axes[1, 0].grid(True, alpha=0.3, axis='y')
axes[1, 0].axhline(y=50, color='red', linestyle='--', alpha=0.5, label='50% baseline')
axes[1, 0].legend()

# Side preference
side_sentiment = pd.crosstab(merged_df['classification'], merged_df['Side'], normalize='index') * 100
side_sentiment.plot(kind='bar', ax=axes[1, 1], color=['#e74c3c', '#3498db'], alpha=0.7, edgecolor='black')
axes[1, 1].set_title('Position Side Preference by Sentiment', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Percentage (%)')
axes[1, 1].set_xlabel('Market Sentiment')
axes[1, 1].legend(title='Side')
axes[1, 1].grid(True, alpha=0.3, axis='y')
plt.setp(axes[1, 1].xaxis.get_majorticklabels(), rotation=0)

plt.tight_layout()
plt.savefig('outputs/trader_behavior.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Trader behavior analysis saved")

# ============================================================================
# SECTION 9: ADVANCED INSIGHTS
# ============================================================================

print("\n" + "="*80)
print("ADVANCED INSIGHTS")
print("="*80)

# Trader segmentation
print("\n👥 TRADER SEGMENTATION:")
trader_performance = merged_df.groupby('Account').agg({
    'Closed PnL': 'sum',
    'Account': 'count'  # number of trades
}).rename(columns={'Account': 'num_trades'})

trader_performance['avg_pnl'] = trader_performance['Closed PnL'] / trader_performance['num_trades']

# Classify traders
trader_performance['category'] = pd.cut(
    trader_performance['Closed PnL'],
    bins=[-np.inf, -1000, 1000, np.inf],
    labels=['Loser', 'Neutral', 'Winner']
)

print(trader_performance['category'].value_counts())

# Profitable trader analysis by sentiment
print("\n💰 TOP TRADERS BY SENTIMENT:")
top_fear_traders = merged_df[merged_df['classification'] == 'Fear'].groupby('Account')['Closed PnL'].sum().nlargest(5)
top_greed_traders = merged_df[merged_df['classification'] == 'Greed'].groupby('Account')['Closed PnL'].sum().nlargest(5)

print("\nTop 5 Traders During FEAR:")
print(top_fear_traders)

print("\nTop 5 Traders During GREED:")
print(top_greed_traders)

In [ ]:
print("\n" + "="*80)
print("KEY FINDINGS SUMMARY")
print("="*80)

findings = f"""
1. SENTIMENT DISTRIBUTION:
   - Fear days: {sentiment_pct.get('Fear', 0):.1f}%
   - Greed days: {sentiment_pct.get('Greed', 0):.1f}%
   - Extreme Fear days: {sentiment_pct.get('Extreme Fear', 0):.1f}%
   - Extreme Greed days: {sentiment_pct.get('Extreme Greed', 0):.1f}%
   - Neutral days: {sentiment_pct.get('Neutral', 0):.1f}%

2. PROFITABILITY BY SENTIMENT:
   - Average PnL (Fear): ${fear_data['Closed PnL'].mean():.2f}
   - Average PnL (Greed): ${greed_data['Closed PnL'].mean():.2f}
   - Statistical significance (Fear vs Greed PnL): {'Yes (p<0.05)' if p_value_pnl < 0.05 else 'No (p≥0.05)'}

3. TRADING VOLUME:
   - Total volume (Fear): ${fear_data['trade_value'].sum():,.0f}
   - Total volume (Greed): ${greed_data['trade_value'].sum():,.0f}

4. WIN RATE:
   - Win rate (Fear): {fear_data['is_profitable'].mean() * 100:.2f}%
   - Win rate (Greed): {greed_data['is_profitable'].mean() * 100:.2f}%

5. POSITION BIAS:
   - Side preference is {'significantly' if p_value_chi < 0.05 else 'not significantly'} associated with sentiment (Chi-square p-value: {p_value_chi:.4f})
"""

print(findings)

# Save summary to file
with open('csv_files/analysis_summary.txt', 'w') as f:
    f.write(findings)

print("\n✓ Analysis summary saved to csv_files/analysis_summary.txt")